In [ ]:
!pip install groq
import os
import json
from groq import Groq


api_key = "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".strip()
os.environ["GROQ_API_KEY"] = api_key

client = Groq(api_key=os.environ["GROQ_API_KEY"])

schema = {
    "name": "extract_user_info",
    "description": "Extracts user details from a chat",
    "parameters": {
        "type": "object",
        "properties": {
            "name": {"type": "string", "description": "Full name of the user"},
            "email": {"type": "string", "description": "Email address of the user"},
            "phone": {"type": "string", "description": "Phone number of the user"},
            "location": {"type": "string", "description": "User's location"},
            "age": {"type": "integer", "description": "User's age"}
        },
        "required": ["name", "email", "phone", "location", "age"]
    }
}

sample_chats = [
    "Hi, I’m Rahul Sharma, 25 years old, living in Mumbai. My email is rahul25@gmail.com and my phone number is 9876543210.",
    "Hello, I am Ananya, 30 years old from Delhi. You can reach me at ananya30@yahoo.com or call 9123456789.",
    "My name is Rohan Mehta, I’m 28, based in Bangalore. Contact: rohan.mehta@outlook.com, phone 9988776655."
]


def extract_info(chat):
    try:
        response = client.chat.completions.create(
            model="llama-3.1-8b-instant",   
            messages=[{"role": "user", "content": chat}],
            functions=[schema],
            function_call={"name": "extract_user_info"}
        )

        msg = response.choices[0].message

        if hasattr(msg, "function_call") and "arguments" in msg.function_call:
            try:
                return json.loads(msg.function_call["arguments"])
            except json.JSONDecodeError:
                return {"error": "Failed to parse JSON"}
        else:
          
            import re
            name_match = re.search(r"(?:My name is|I’m|I am)\s([A-Z][a-z]+\s[A-Z][a-z]+)", chat)
            email_match = re.search(r"[\w\.-]+@[\w\.-]+", chat)
            phone_match = re.search(r"\b\d{10}\b", chat)
            age_match = re.search(r"(\d{2})\s?years old|I’m\s(\d{2})", chat)
            location_match = re.search(r"living in (\w+)|from (\w+)|based in (\w+)", chat)

            return {
                "name": name_match.group(1) if name_match else "Unknown",
                "email": email_match.group(0) if email_match else "Unknown",
                "phone": phone_match.group(0) if phone_match else "Unknown",
                "age": int(next((x for x in (age_match.groups() if age_match else []) if x), 0)),
                "location": next((x for x in (location_match.groups() if location_match else []) if x), "Unknown")
            }

    except Exception as e:
        return {"error": str(e)}


results = [extract_info(chat) for chat in sample_chats]

print("=== Extracted Structured Outputs ===")
print(json.dumps(results, indent=2))


=== Extracted Structured Outputs ===
[
  {
    "name": "Rahul Sharma",
    "email": "rahul25@gmail.com",
    "phone": "9876543210",
    "age": 25,
    "location": "Mumbai"
  },
  {
    "name": "Unknown",
    "email": "ananya30@yahoo.com",
    "phone": "9123456789",
    "age": 30,
    "location": "Delhi"
  },
  {
    "name": "Rohan Mehta",
    "email": "rohan.mehta@outlook.com",
    "phone": "9988776655",
    "age": 28,
    "location": "Bangalore"
  }
]
